In [2]:
import numpy as np
import os
from PIL import Image
import helper
import json
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# training data path

train_normal_path = "./../../../datasets/rosacea-detection/train/normal"
train_rosacea_path = "./../../../datasets/rosacea-detection/train/rosacea"

In [4]:
X_normal = np.zeros((512 * 512 * 3, 500))
X_rosacea = np.zeros((512 * 512 * 3, 250))

print("Loading X_normal....")
for i, filename in zip(range(500),os.listdir(train_normal_path)):
    filepath = os.path.join(train_normal_path, filename)
    img = (np.array(Image.open(filepath)) ).flatten()
    X_normal[:, i] = img.copy()
    # print(f"{i}th normal loaded")
print(f"X_normal loaded, shape of {np.shape(X_normal)}")

print("Loading X_rosacea....")
for i, filename in zip(range(250),os.listdir(train_rosacea_path)):
    filepath = os.path.join(train_rosacea_path, filename)
    img = (np.array(Image.open(filepath)) ).flatten()
    X_rosacea[:, i] = img.copy()
    # print(f"{i}th rosacea loaded")
print(f"X_rosacea loaded, shape of {np.shape(X_rosacea)}")

Loading X_normal....
X_normal loaded, shape of (786432, 500)
Loading X_rosacea....
X_rosacea loaded, shape of (786432, 250)


In [5]:
X_train = np.hstack((X_normal, X_rosacea)).transpose()
print(np.shape(X_train))

(750, 786432)


In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)

In [ ]:
Y_train = np.zeros(750)
Y_train[500:] = 1
# normal is 0
# rosacea is 1

In [ ]:
val_normal_path = "./../../../datasets/rosacea-detection/val/normal"
val_rosacea_path = "./../../../datasets/rosacea-detection/val/rosacea"
test_normal_path = "./../../../datasets/rosacea-detection/test/normal"
test_rosacea_path = "./../../../datasets/rosacea-detection/test/rosacea"

for k in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15}:
    knn = KNeighborsClassifier(n_neighbors = k, metric = 'cosine')
    knn.fit(X_train_std, Y_train)
    TP_v, TN_v, FP_v, FN_v = 0, 0, 0, 0
    TP_t, TN_t, FP_t, FN_t = 0, 0, 0, 0
    count = 0
    print(f"k is {k}")
    # Validation
    for filename in os.listdir(val_normal_path):
        file_path = os.path.join(val_normal_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            TN_v += 1
        else:
            FP_v += 1
        count += 1
        print(count)

    for filename in os.listdir(test_rosacea_path):
        file_path = os.path.join(test_rosacea_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            FN_v += 1
        else:
            TP_v += 1
        count += 1
        print(count)
        
    # Test
    for filename in os.listdir(test_normal_path):
        file_path = os.path.join(test_normal_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            TN_t += 1
        else:
            FP_t += 1
        count += 1
        print(count)
        
    for filename in os.listdir(test_rosacea_path):
        file_path = os.path.join(test_rosacea_path, filename)
        sample = np.array(Image.open(file_path)) 
        if knn.predict(sc.transform(sample.flatten().reshape(1, -1))) == 0:
            FN_t += 1
        else:
            TP_t += 1
        count += 1
        print(count)
    # Output result
    accuracy_v, precision_v, recall_v, F1_v = helper.get_performance_metrics(
        TP_v, TN_v, FP_v, FN_v)
    
    accuracy_t, precision_t, recall_t, F1_t = helper.get_performance_metrics(
        TP_t, TN_t, FP_t, FN_t)

    result = {
        "unequal" : 1,
        "n_neighbors" :k,
        "metric" : "cosine",
        "p" : None,
        "TN_v" : TN_v,
        "TP_v" : TP_v,
        "FP_v" : FP_v,
        "FN_v" : FN_v,
        "accuracy_v" : accuracy_v,
        "precision_v" : precision_v,
        "recall_v" : recall_v,
        "F1_v" : F1_v,

        "TN_t" : TN_t,
        "TP_t" : TP_t,
        "FP_t" : FP_t,
        "FN_t" : FN_t,
        "accuracy_t" : accuracy_t,
        "precision_t" : precision_t,
        "recall_t" : recall_t,
        "F1_t" : F1_t
    }

    json_object = json.dumps(result, indent = 4)

    # writing to file
    # json_name = "p="+str(p)+"-n_neighbors=" + str(k)+".json"
    json_name = f"knn-metric-cosine-n_neighbors={k}.json"
    json_path = os.path.join("./../results",json_name)
    with open(json_path,'w') as outfile:
        outfile.write(json_object)
